# Listen to your customers: Enhance App Reviews with ChatGPT API

#### Rafael Buck

As a data science, product, business, or marketing person, you understand the significance of app store reviews. They provide valuable insights and shape the success of your app. In this notebook, we'll guide you through three powerful steps to leverage the ChatGPT API and transform the way you handle app store reviews. Let's get started!

## Step 1: Gathering App Store Reviews

In [ ]:
!pip install app_store_scraper

In [ ]:
from app_store_scraper import AppStore
import pandas as pd
import numpy as np

In [ ]:
app_name = "YOUR_APP_NAME"  # Replace with your app's name
app_id = "YOUR_APP_ID"  # Replace with your app's ID

reviews = AppStore(country='us', 
                   app_name=app_name, 
                   app_id=app_id)

reviews.review(how_many=1500)

In [ ]:
tmp = pd.DataFrame(np.array(reviews.reviews),columns=['review'])
df = tmp.join(pd.DataFrame(tmp.pop('review').tolist()))
df.head()

## Step 2: Summarizing Bad Reviews using ChatGPT API

Now, let's harness the power of the ChatGPT API to automatically summarize negative reviews. With OpenAI's Python library, we can achieve this easily. Take a look at the code snippet below:

In [ ]:
!pip install openai
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv # Save your key in a variable in .env file

In [ ]:
load_dotenv() # read local .env file
API_TOKEN = os.getenv('OPENAI_API_KEY')

In [ ]:
threshold_date = pd.to_datetime('2023-01-01')
df['rating'] = df['rating'].astype(int)

bad_reviews = df[df['rating']<3] # getting < 3 rating reviews
bad_reviews = bad_reviews[bad_reviews['date'] > threshold_date].reset_index() # getting most recent ones

summaries = []
types = []
suggestions = []

In [ ]:
bad_reviews.head()

In [ ]:
bad_reviews = bad_reviews.head(50).reset_index() # getting just 50 reviews

In [ ]:
import openai
openai.api_key = API_TOKEN

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
import json

for index, row in bad_reviews.iterrows():

    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from a social sharing content tech app. 
    
    The review is delimited with triple backticks. \
    Format your response as a JSON object with \
    "Summary", "Type" and "Feature suggestion" as the keys.
    If the information isn't present, use "unknown" \
    as the value.
    Make your response as short as possible for "Summary" and "Feature suggestion" keys.\
    Format the "Type" value using at most 2 words depending on \
    the problem exposed in the review.\
    Make sure that all responses are made in English.
    
    Review text: '''{row['review']}'''
    """

    print(prompt)

    response = get_completion(prompt)
    print(index, response, "\n")

    json_response = json.loads(response)
    
    summary = json_response['Summary']
    summaries.append(summary)

    type = json_response['Type']
    types.append(type)

    suggestion = json_response['Feature suggestion']
    suggestions.append(suggestion)

print(summaries)

## Step 3: Identifying Insights and A/B Testing

Armed with the summarized negative reviews, we can now gain valuable insights into the pain points, usability issues, or feature requests. Analyze the patterns in the summaries to understand the reasons behind negative feedback.

To address these concerns effectively, consider implementing A/B testing. By creating two versions of your app—one with improvements based on user feedback—and randomly assigning users to each version, you can measure the impact of changes. Monitor user engagement, satisfaction, and other relevant metrics to gauge the effectiveness of your improvements.

Remember, listening to your customers is paramount to your app's success. The ChatGPT API empowers you to extract meaningful insights efficiently, allowing you to make data-driven decisions and enhance your product accordingly.

In [ ]:
!pip install wordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Sample text data
text_data = text_data = ' '.join(types)
# Create a WordCloud object
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_data)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Feel free to copy and paste this code into a Jupyter Notebook, and make sure to replace the placeholder values (**YOUR_APP_ID**, **YOUR_APP_NAME** and **OPENAI_API_KEY**) with the appropriate values for your app and OpenAI API key.

Enjoy leveraging the ChatGPT API to revolutionize your app store review process!